<a href="https://colab.research.google.com/github/edabezek/MachineLearning/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
print('numpy:', numpy.__version__)

import scipy
print('scipy:', scipy.__version__)

import matplotlib
print('matplotlib:', matplotlib.__version__)

import IPython
print('iPython:', IPython.__version__)

import sklearn
print('scikit-learn:', sklearn.__version__)

numpy: 1.19.5
scipy: 1.4.1
matplotlib: 3.2.2
iPython: 5.5.0
scikit-learn: 0.22.2.post1


In [ ]:
import keras
print('keras: ', keras.__version__)

# optional
import theano
print('Theano: ', theano.__version__)

import tensorflow as tf
print('Tensorflow: ', tf.__version__)

keras:  2.6.0
Theano:  1.1.2
Tensorflow:  2.6.0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
veriler = pd.read_csv('Churn_Modelling.csv')
veriler.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#veri on isleme

X= veriler.iloc[:,3:13].values #rownumber,customerıd,surname atıyoruz
Y = veriler.iloc[:,13].values #son kolon Y değeri olacak



In [ ]:
#encoder: Kategorik -> Numeric
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
X[:,1] = le.fit_transform(X[:,1]) #x'in 1 inci kolonu encode olup yerine geçecek, cografi kısım

X[:5]

array([[619, 0, 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 2, 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 0, 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 0, 'Female', 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 2, 'Female', 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [ ]:
le2 = preprocessing.LabelEncoder()
X[:,2] = le2.fit_transform(X[:,2]) #gender encoding olacak

In [ ]:
X[:5]

array([[619, 0, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 2, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 0, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 0, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 2, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]], dtype=object)

In [ ]:
#burada ülke kolonunu 1-0 şeklinde onehotencoding yapıyoruz. gender zaten 1-0 olmuştu.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ohe = ColumnTransformer([("ohe", OneHotEncoder(dtype=float),[1])],
                        remainder="passthrough"
                        )
X = ohe.fit_transform(X)
X = X[:,1:]


In [ ]:
X[:5]

array([[0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [ ]:
veriler.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#verilerin egitim ve test icin bolunmesi
from sklearn.model_selection import train_test_split

x_train, x_test,y_train,y_test = train_test_split(X,Y,test_size=0.33, random_state=0)


In [ ]:
#verilerin olceklenmesi
from sklearn.preprocessing import StandardScaler

sc=StandardScaler() #burada standart dağılıma göre düzeltme yaptık hoca soru cevap kısmında anlatacakmış.

X_train = sc.fit_transform(x_train)
X_test = sc.fit_transform(x_test)

In [ ]:
#Yapay Sinir ağı 
import keras
from keras.models import Sequential
from keras.layers import Dense

#relu : rectifier, 0 ın altında 0 , 0 ın üstünde lieer artan bir fonksiyon
#problem linear seperable : doğrusal olarak birbirinden ayrılabiliyorsa o zaman gizli katman kullanmamıza gerek yok
#çıkış katmanımızda 1 tane nöron var , giriş katmanında 11  tane nöron var
#6 dan başlamamızın sebebi 11+1=12/2 yaparak sallamamız (hoca sanat dedi)

classifier = Sequential()
#burada init uniform dememizin sebebi gradient ten dolayı 0 a yakın birer veri olarak sinapsislerin üzrine belirlemek
#daha sonra sinir ağı çalıştıkça bu değişecek
#input_dim  kaç değişken olduğu
classifier.add(Dense(6, kernel_initializer='uniform', activation = 'relu' , input_dim = 11)) #6 nöronlu gizli katman var

classifier.add(Dense(6, kernel_initializer='uniform', activation = 'relu')) #bir katman daha ekledik

classifier.add(Dense(1, kernel_initializer='uniform', activation = 'sigmoid'))#çıkış katmanı

In [ ]:
#eğitme kısmı
#adam sinapsisler üzerindeki değerlerin nasıl optimize edileceğini belirliyor.
#loss : çıkan hatanın nasıl hesaplancağını belirtiyor. binary olmasının sebebi Ynin 1 veya 0 değerini alması
classifier.compile(optimizer = 'adam', loss =  'binary_crossentropy' , metrics = ['accuracy'] )

#çalıştırma kısmı
classifier.fit(X_train, y_train, epochs=50)

#tahmin kısmı
y_pred = classifier.predict(X_test)


Epoch 1/50
210/210 [==============================] - 1s 2ms/step - loss: 0.5884 - accuracy: 0.7976
Epoch 2/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4380 - accuracy: 0.7979
Epoch 3/50
210/210 [==============================] - 0s 1ms/step - loss: 0.4303 - accuracy: 0.7979
Epoch 4/50
210/210 [==============================] - 0s 1ms/step - loss: 0.4269 - accuracy: 0.7979
Epoch 5/50
210/210 [==============================] - 0s 1ms/step - loss: 0.4245 - accuracy: 0.7979
Epoch 6/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4214 - accuracy: 0.7979
Epoch 7/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4199 - accuracy: 0.7979
Epoch 8/50
210/210 [==============================] - 0s 1ms/step - loss: 0.4177 - accuracy: 0.8051
Epoch 9/50
210/210 [==============================] - 0s 1ms/step - loss: 0.4167 - accuracy: 0.8219
Epoch 10/50
210/210 [==============================] - 0s 1ms/step - loss: 0.4156 - accuracy: 0.8233

In [ ]:
y_pred[:5] #sonuçlar yüzdeli olduğundan hoca yüzde 50 sini altı ve üstü diye bölecek
#böylece müşteri bıraktı yada bırakmadı cinsinden gerçek veriyle(y_test) karşılaştıracağız.

array([[0.2620251 ],
       [0.30824918],
       [0.18529812],
       [0.05668536],
       [0.19243503]], dtype=float32)

In [ ]:
y_test[:5]

array([0, 1, 0, 0, 0])

In [ ]:
y_pred = (y_pred > 0.5) # true 1 ,falce 0 olacak

In [ ]:
y_pred[:5]

array([[False],
       [False],
       [False],
       [False],
       [False]])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

print(cm)


[[2561   56]
 [ 472  211]]


2561+211= doğru tahmin